# Importing libraries

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding,LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np



In [3]:
file=open("data for next word prediction.txt","r",encoding='utf8')

# Storing file in list
lines=[]
for each in file:
    lines.append(each)

# Converting list in string and then in lowercase and removing punctuation
data=" ".join(lines)
data=data.lower()
data=data.replace(',','').replace('\\','').replace("?","").replace("-","").replace(".","")
print(data)

the project gutenberg ebook of pride and prejudice
     
 this ebook is for the use of anyone anywhere in the united states and
 most other parts of the world at no cost and with almost no restrictions
 whatsoever you may copy it give it away or reuse it under the terms
 of the project gutenberg license included with this ebook or online
 at wwwgutenbergorg if you are not located in the united states
 you will have to check the laws of the country where you are located
 before using this ebook
 
 title: pride and prejudice
 
 author: jane austen
 
 release date: june 1 1998 [ebook #1342]
                 most recently updated: october 29 2024
 
 language: english
 
 credits: chuck greif and the online distributed proofreading team at http://wwwpgdpnet (this file was produced from images available at the internet archive)
 
 
 *** start of the project gutenberg ebook pride and prejudice ***
                             [illustration:
 
                              george allen
        

# Text Preprocessing
## Tokenization

In [4]:
from nltk.tokenize import WordPunctTokenizer
tokenizer=WordPunctTokenizer()
tokens=tokenizer.tokenize(data)

In [5]:
tokens[0]

'the'

In [6]:
# Getting unique words and word_index
print("Total words in data are: ",len(tokens))
vocabulory=list(set(tokens))
vocabulory_size=len(vocabulory)
word_index={}
index_word={}
for index,word in enumerate(vocabulory):
    word_index[word]=index+1                    # 0 is reserved for padding
    index_word[index+1]=word

print("Total Unique words in data are",len(word_index)+1) # +1 for the padding/unknown token
word_index

Total words in data are:  42963
Total Unique words in data are 5914


{'customers': 1,
 'promise': 2,
 'appears': 3,
 'piles': 4,
 'revision': 5,
 'handsome': 6,
 'blinds': 7,
 'locality': 8,
 'beechcraft': 9,
 'harmony': 10,
 'stockings': 11,
 'retirement': 12,
 'conjecturing': 13,
 'gaping': 14,
 'bushpilot': 15,
 'warsawyes': 16,
 'taking': 17,
 'access': 18,
 'helplessness': 19,
 'docketing': 20,
 'barely': 21,
 'cunning': 22,
 'lives': 23,
 'thomson_': 24,
 'fiction': 25,
 'overbearing': 26,
 'loss': 27,
 'aircraft': 28,
 'promises': 29,
 'companions': 30,
 'lvi': 31,
 'perishable': 32,
 'reserved': 33,
 'agreed': 34,
 'custom': 35,
 'highs': 36,
 'result': 37,
 'terminated': 38,
 'irene': 39,
 'signature': 40,
 'goodwill': 41,
 'engaged': 42,
 'brick': 43,
 'seventeen': 44,
 'persons': 45,
 'fiery': 46,
 'necessitate': 47,
 'nationalities': 48,
 'maunders': 49,
 'tailpiece': 50,
 'remains': 51,
 'compare': 52,
 '515': 53,
 'distinction': 54,
 'tide': 55,
 'located': 56,
 'territory': 57,
 '198': 58,
 'seldom': 59,
 'sits': 60,
 'selflighting': 61,


DefualtDict: if you try to access a key that doesn't exist, it automatically creates that key and assigns a default value to it, instead of raising a KeyError.

In [7]:
# Convert the tokens to numerical sequences
from collections import defaultdict
word_to_index = defaultdict(lambda: 0) # By default it will assign 0 to unknown words
numerical_sequence = [word_index[token] for token in tokens]

print("\nNumerical Sequence of Tokens (Partial):\n", numerical_sequence[:10])


Numerical Sequence of Tokens (Partial):
 [4398, 648, 5494, 1020, 1818, 2730, 2139, 2548, 1256, 1020]


# Creating Training sequences(Input-Output Pairs)
This is the core step for sequence prediction. We'll create overlapping sequences of words and their immediate next word.

When creating N-gram sequences for next-word prediction, you always have one less sequence than the total number of words.

The first word (word_1) cannot be used as an input sequence to predict word_2, because it doesn't have a preceding word.

In [8]:
input_sequences=[]
for i in range(1,len(numerical_sequence)):
    n_gram_sequence=numerical_sequence[:i+1]
    input_sequences.append(n_gram_sequence)

print("\nRaw N-Gram Input Sequences (Partial):\n", input_sequences[:5])
print("Total number of N-Gram Sequences:", len(input_sequences))


Raw N-Gram Input Sequences (Partial):
 [[4398, 648], [4398, 648, 5494], [4398, 648, 5494, 1020], [4398, 648, 5494, 1020, 1818], [4398, 648, 5494, 1020, 1818, 2730]]
Total number of N-Gram Sequences: 42962


# Padding

In [9]:
max_length_word=max([len(word) for word in vocabulory])
print(max_length_word)
padding=pad_sequences(input_sequences,maxlen=max_length_word,padding='pre')
padding

29


array([[   0,    0,    0, ...,    0, 4398,  648],
       [   0,    0,    0, ..., 4398,  648, 5494],
       [   0,    0,    0, ...,  648, 5494, 1020],
       ...,
       [3685, 2340,  998, ..., 5830, 3917, 3781],
       [2340,  998, 4375, ..., 3917, 3781, 4398],
       [ 998, 4375, 3140, ..., 3781, 4398, 4318]])

In [10]:
from tensorflow.keras.preprocessing.text import one_hot
vocabulory=str(vocabulory)
str_vocabulory=" ".join(vocabulory)
str_vocabulory=str_vocabulory.replace("]"," ").replace("[","")
one_hot_reprst=one_hot(vocabulory,10)
one_hot_reprst[0]


2

# Obtaining One-Hot Vectors

In [11]:
# 5. Use tf.keras.utils.to_categorical to get the one-hot vector
one_hot_vector = tf.keras.utils.to_categorical(one_hot_reprst, num_classes=vocabulory_size)
print(one_hot_vector.astype(int))

[[0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Neural Networks: Creating Model and assigning layers

In [12]:
model=Sequential()
model.add(Embedding(input_dim=vocabulory_size,output_dim=20,input_length=max_length_word))
model.add(Dense(vocabulory_size,activation='softmax'))
model.add(LSTM(150)) # Having 150 neurons
print("Model Summary\n")
model.summary()
print(vocabulory_size) # output:5913
print(max_length_word) # output: 29

e:\vs code software\python\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model Summary



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

5913
29


In [13]:
# Remember: input_length for the Embedding layer is max_sequence_len - 1
# because the last word of the padded sequence is the label (y).
model_input_length = max_length_word - 1 # So, 28 in our case

model = Sequential()
# 1. Embedding Layer: Takes word indices, outputs word vectors (3D)
model.add(Embedding(input_dim=vocabulory_size, output_dim=100, input_length=model_input_length))

# 2. LSTM Layer: Processes the sequence of word vectors (3D input), outputs a single vector (2D)
# By default, LSTM outputs the last hidden state (2D), which is what Dense expects.
model.add(LSTM(150)) # Having 150 neurons

# 3. Dense Layer: Takes the LSTM's output, outputs probabilities for each word (2D)
model.add(Dense(vocabulory_size, activation='softmax'))

print("Model Summary\n")
model.summary()

print(f"Vocabulary Size: {vocabulory_size}")
print(f"Max Sequence Length (for padding): {max_length_word}")
print(f"Model Input Length (for Embedding): {model_input_length}")

Model Summary



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Vocabulary Size: 5913
Max Sequence Length (for padding): 29
Model Input Length (for Embedding): 28


# Compiling and Training model

In [24]:
# Compiling the Model
model.compile(loss='sparse_categorical_crossentropy', optimizer= 'adam', metrics=['accuracy']) # loss is made this to avoid memory error in x and y separation step
print("Model compiled successfully")

Model compiled successfully


In [26]:
# Separating in X and y
# Separate features (X) and labels (y)
X = padding[:, :-1] # All words except the last one
y= padding[:, -1] # The last word (the target)
print(len(X))

42962


In [29]:
# Training the model
model.fit(X,y,epochs=10, verbose=1, batch_size=64) # epochs: number of times the model will go through the entire dataset   
# The error here occurs because of the cpu but if i do this on gpu(google colab) it works fine
# The error is at lowerlevel and is related to the tensorflow library and cpu 

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sequential_1_1/lstm_1_1/while/TensorListPushBack_13 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "e:\vs code software\python\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "e:\vs code software\python\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "e:\vs code software\python\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "e:\vs code software\python\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "e:\vs code software\python\Lib\asyncio\base_events.py", line 641, in run_forever

  File "e:\vs code software\python\Lib\asyncio\base_events.py", line 1987, in _run_once

  File "e:\vs code software\python\Lib\asyncio\events.py", line 88, in _run

  File "e:\vs code software\python\Lib\site-packages\ipykernel\kerne